# 네이버 웹툰 크롤링하기

## 필요한 라이브러리 호출하기

In [ ]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from time import sleep
from tqdm import tqdm
import time
import re
import requests
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from io import BytesIO

## 연재 중인 네이버 웹툰 정보 크롤링

**함수 만들기**

In [ ]:
def naver_crawling(title_list, URL):
    driver=webdriver.Chrome('chromedriver.exe')
    driver.get(URL)

    img_list=[] 
    artist_list=[] 
    genre_list=[] 
    score_list=[]  
    story_list=[] 

    for i in range(len(title_list)):

        time.sleep(0.5)

        # 제목에 해당하는 엘리먼트 클릭 순서대로 클릭
        page=driver.find_elements_by_class_name('title')
        page[i].click()

        time.sleep(0.01)

        #셀레니움으로 열어서 페이지 정보 가져오기
        html = driver.page_source
        soup = bs(html,'html.parser')

        #작품 썸네일 이미지
        img=soup.select('#content > div.comicinfo > div.thumb > a > img')[0]['src']
        img_list.append(img)

        #작가님 닉네임 수집
        artist = soup.select('#content > div.comicinfo > div.detail > h2 > span.wrt_nm')
        artist_list.append(artist[0].string.strip())

        #줄거리 링크 
        story=soup.select('#content > div.comicinfo > div.detail > p:nth-child(2)')
        story_list.append(story[0].get_text())

        #작품 장르 수집
        genre = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.genre')
        genre_list.append(genre[0].string)

        # 최신 별점 평균 점수 수집 (최대 10화 분량) : 이부분은 참조함
        # https://prod.velog.io/@2taesung/wt.gg-%EB%84%A4%EC%9D%B4%EB%B2%84-%EC%9B%B9%ED%88%B0-%ED%81%AC%EB%A1%A4%EB%A7%81
        score = soup.find_all('strong')

        scorelist=[] ; ii=9
        while score[ii].text[0].isnumeric()==True:
            scorelist.append(float(score[ii].text))
            ii +=1
            if len(scorelist) == 3:
              break
        score_list.append(sum(scorelist)/len(scorelist))

        time.sleep(0.5)

        driver.back()
        
        time.sleep(0.2)
        
        page.clear()
        
        time.sleep(0.01)
        
        #데이터 프레임으로 변환
        df = pd.DataFrame(zip(title_list, artist_list, genre_list, score_list, story_list,img_list, link_list), 
                             columns = ['Title', 'Artist', 'Genre', 'Score(recent 10)', 'Story', 'Image', 'Link'])
    return df

**연재 중 웹툰 크롤링하기**

In [ ]:
URL = 'https://comic.naver.com/webtoon/weekday.nhn'
html = requests.get(URL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
soup = bs(html, 'html.parser')

**링크 주소와 제목 가져오기**

In [ ]:
link_list = []
title_list = []

#각 숫자 1~7까지는 월화수목금토일을 의미한다.
for i in range(1,8): 
    tl_tag=soup.select(f'#content > div.list_area.daily_all > div:nth-child({i}) > div > ul > li > a')
    
    #각 요일의 각 웹툰들에 해당하는 주소와 제목을 가져온다.
    for j in range(len(tl_tag)): 
        title = tl_tag[j]['title']
        link = tl_tag[j]['href']
        title_list.append(title)
        link_list.append(f'https://comic.naver.com{link}')


**가져온 제목을 기반으로 필요한 정보 수집하기**

In [ ]:
URL='https://comic.naver.com/webtoon/weekday.nhn'
df_yonjae  = naver_crawling(title_list, URL)

**데이터 프레임으로 변환하고 csv로 저장하기**

In [ ]:
df_yonjae.to_csv('naver_webtoon_real.csv', index=False)

## 완결 웹툰 크롤링

In [ ]:
def naver_crawling_finish(title_list, URL):
    driver=webdriver.Chrome('chromedriver.exe')
    driver.get(URL)
    
    img_list=[] 
    artist_list=[] 
    genre_list=[] 
    score_list=[]  
    story_list=[] 

    for i in tqdm(range(1,1283)):
        page = driver.find_elements_by_class_name('thumb')
        
        time.sleep(0.5)

        # 제목에 해당하는 엘리먼트 클릭 순서대로 클릭
        page[i].click()

        time.sleep(0.01)

        #셀레니움으로 열어서 페이지 정보 가져오기
        html = driver.page_source
        soup = bs(html,'html.parser')

        #작품 썸네일 이미지
        img=soup.select('#content > div.comicinfo > div.thumb > a > img')[0]['src']

        img_list.append(img)

        #작가님 닉네임 수집
        artist = soup.select('#content > div.comicinfo > div.detail > h2 > span.wrt_nm')
        artist_list.append(artist[0].string.strip())

        #줄거리 링크 
        story=soup.select('#content > div.comicinfo > div.detail > p:nth-child(2)')
        story_list.append(story[0].get_text())

        #작품 장르 수집
        genre = soup.select('#content > div.comicinfo > div.detail > p.detail_info > span.genre')
        genre_list.append(genre[0].string)

        # 최신 별점 평균 점수 수집 (최대 10화 분량) : 이부분은 참조함
        # https://prod.velog.io/@2taesung/wt.gg-%EB%84%A4%EC%9D%B4%EB%B2%84-%EC%9B%B9%ED%88%B0-%ED%81%AC%EB%A1%A4%EB%A7%81
        score = soup.find_all('strong')

        scorelist=[] ; ii=9
        while score[ii].text[0].isnumeric()==True:
            scorelist.append(float(score[ii].text))
            ii +=1
            if len(scorelist) == 3:
              break
        score_list.append(sum(scorelist)/len(scorelist))

        time.sleep(0.5)

        driver.back()

        time.sleep(0.2)

        page.clear()

        time.sleep(0.01)
        
    df = pd.DataFrame(zip(artist_list, genre_list, score_list, story_list,img_list), 
                      columns = ['Artist', 'Genre', 'Score(recent 10)', 'Story', 'Image'])
    return df

**완결 웹툰 크롤링하기**

In [ ]:
URL = 'https://comic.naver.com/webtoon/finish'
html = requests.get(URL).text # html 문서 전체를 긁어서 출력해줌, .text는 태그 제외하고 text만 출력되게 함
soup = bs(html, 'html.parser')

**링크 주소와 제목 가져오기**

In [ ]:
link_list = []
title_list = []

for i in tqdm(range(1,1283)): #완결 웹툰 가장 마지막 인덱스(현재는 크롤링 날짜 기준)
    tl_tag=soup.select(f'#content > div.list_area > ul > li:nth-child({i}) > dl > dt > a')
    title = tl_tag[0]['title']
    link = tl_tag[0]['href']
    
    title_list.append(title)
    link_list.append(f'https://comic.naver.com{link}')

**링크 주소, 제목 데이터 프레임으로 만들고 csv로 저장하기**

In [ ]:
df_final_tl = pd.DataFrame(zip(title_list,link_list), columns = ['Title', 'Link'])
df_final_tl.to_csv('naver_webtoon_final_title_link.csv', index=False)

**저장된 데이터 프레임 불러와 확인 후, 제목 리스트 뽑아 함수에 넣어 크롤링 진행하기**

In [ ]:
# 백업하기
df_final_tl = pd.read_csv('naver_webtoon_final_title_link.csv')

In [ ]:
title_list = df_final_tl['Title'].tolist()

**함수 돌리기**

In [ ]:
df_finish = naver_crawling_finish(title_list, URL)

**데이터 프레임으로 변환하고 csv로 저장하기**

In [ ]:
df_finish.to_csv('naver_webtoon_finish_without_tl.csv', index=False)
df_final_etl = pd.read_csv('naver_webtoon_finish_without_tl.csv')

**데이터 프레임 합치고 연재중 웹툰 csv와 순서 맞추기**

In [ ]:
df = pd.concat([df_final_tl,df_final_etl],axis=1, join='inner') 
df=df[['Title', 'Artist', 'Genre', 'Score(recent 10)', 'Story', 'Image', 'Link']]
df.to_csv('naver_webtoon_finish.csv', index=False)

In [ ]:
pd.read_csv('naver_webtoon_finish.csv')

## 네이버 웹툰 회차별 썸네일 크롤링하기
출처: [사이트 링크](https://domdom.tistory.com/entry/%ED%81%AC%EB%A1%A4%EB%A7%81-%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9C%BC%EB%A1%9C-%EB%84%A4%EC%9D%B4%EB%B2%84-%EC%9B%B9%ED%88%B0-%EC%86%8C%EA%B0%9C-%EB%B0%8F-%ED%9A%8C%EC%B0%A8-%EC%A0%95%EB%B3%B4-%EA%B0%80%EC%A0%B8%EC%98%A4%EA%B8%B0)

In [ ]:
df1 = pd.read_csv('naver_webtoon_real.csv')
df2 = pd.read_csv('naver_webtoon_finish.csv')

In [ ]:
df = pd.concat([df1,df2], ignore_index=True)

In [ ]:
# 함수 만들기

def bring_thumb_images(df):
    imglist = []
    cnt=1
    for i in tqdm(range(len(df))):
        
        print(df['Title'][i], '수집 중')
        
        # 웹툰의 각 회차별 썸네일 가져오기
        page = 1
        imgSrc=[]
        while True:
            url = df['Link'][i]+f"&page={str(page)}"
            res = requests.get(url)
            soup = bs(res.text, 'lxml')

            viewList = soup.select('.viewList tr')


            for view in viewList[3:]: # 배너 제외
                imgSrc.append(view.select_one('a > img').attrs.get('src'))
                cnt += 1
            time.sleep(0.01)

            if soup.select_one('a.next') == None:
                print('last page was ' + str(page))
                print(f'1830 중 {i+1} 완료')
                break
            page = page + 1
            time.sleep(0.01)
        
        #위에서 만들어진 리스트를 원하는 형태로 바꾸어 하나의 리스트로 담아주기
        img_str = " π ".join(imgSrc)
        imglist.append(img_str)
    
    print(cnt)
    
    return imglist

In [ ]:
img = bring_thumb_images(df)

In [ ]:
df_thumb = pd.DataFrame({'Thumbs': img })
df_thumb.to_csv('naver_webtoon_thumbs.csv', index=False)

df = pd.concat([df, df_thumb], axis=1)
df.to_csv('naver_webtoon_all_with_thumbs.csv', index=False)